<a href="https://colab.research.google.com/github/minhasplanilhas/n8n/blob/main/05_Agentes_com_GPT_4o_Mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --q dotenv
!pip install --q crewai
!pip install --q 'crewai[tools]'

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, DallETool
from langchain_openai import ChatOpenAI

In [ ]:
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Inicializar o modelo OpenAI
gpt_4o_mini = ChatOpenAI(model_name='gpt-4o-mini')


#TOOLS

In [ ]:
# Ferramenta de pesquisa na internet
search_tool = SerperDevTool()

In [ ]:
# Ferramenta DALL-E com especificação de tamanho 16:9
dalle_tool = DallETool(size='1792x1024')  # Formato 16:9

#AGENTES

 - Pesquisador da web
 - Escritor de título
 - Roterista
 - Especialista em SEO
 - Engenheiro de prompts para imagens
 - Design de imagens
 - Revisor


#PESQUISADOR

In [ ]:
pesquisador = Agent(
    role='Pesquisador de Tema',
    goal='Pesquisar informações detalhadas sobre {tema} na internet',
    backstory=(
        "Você é um especialista em pesquisa, com habilidades aguçadas para "
        "encontrar informações valiosas e detalhadas na internet."
    ),
    verbose=True,
    memory=True,
    tools=[search_tool],
    llm=gpt_4o_mini
)


In [ ]:
tarefa_pesquisa = Task(
    description=(
        "Pesquisar informações detalhadas e relevantes sobre o tema: {tema}. "
        "Concentre-se em aspectos únicos e dados importantes que podem enriquecer o vídeo. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Um documento com as principais informações e dados sobre {tema}.',
    tools=[search_tool],
    agent=pesquisador
)

#ESCRITOR TÍTULO

In [ ]:
# 2. Agente Escritor de Títulos
escritor_titulos = Agent(
    role='Escritor de Títulos de Vídeo',
    goal='Criar títulos atraentes e otimizados para vídeos sobre {tema}',
    verbose=True,
    memory=True,
    backstory=(
        "Você tem uma habilidade especial para criar títulos que capturam a "
        "essência de um vídeo e atraem a atenção do público."
    ),
    llm=gpt_4o_mini
)

In [ ]:
# 3. Agente Escritor de Roteiro
escritor_roteiro = Agent(
    role='Escritor de Roteiro',
    goal='Escrever um roteiro detalhado e envolvente para um vídeo sobre {tema}',
    verbose=True,
    memory=True,
    backstory=
        f"""Você é um contador de histórias talentoso, capaz de transformar
        informações em narrativas cativantes para vídeos."""
    ,
    llm=gpt_4o_mini
)


In [ ]:
# 4. Agente Especialista em SEO para YouTube
especialista_seo = Agent(
    role='Especialista em SEO para YouTube',
    goal='Otimizar o roteiro e o título para que o vídeo tenha uma alta performance no YouTube',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um expert em SEO, com um profundo entendimento das melhores "
        "práticas para otimizar conteúdo para o YouTube."
    ),
    llm=gpt_4o_mini
)

In [ ]:
# 5. Agente Criador de Prompts para DALL-E
criador_prompt_dalle = Agent(
    role='Criador de Prompts para DALL-E',
    goal='Escrever um prompt para gerar uma imagem usando DALL-E com base no tema do vídeo {tema}',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um especialista em criar descrições detalhadas e imaginativas que "
        "permitem ao DALL-E gerar imagens impressionantes com base em textos."
    ),
    llm=gpt_4o_mini
)

In [ ]:
# 6. Agente Gerador de Imagens com DALL-E
gerador_imagens = Agent(
    role='Gerador de Imagens com DALL-E',
    goal='Gerar uma imagem usando DALL-E com o prompt fornecido pelo Criador de Prompts para DALL-E',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um mestre em transformar descrições textuais em belas imagens, utilizando o poder do DALL-E."
    ),
    tools=[dalle_tool],
    llm=gpt_4o_mini
)


In [ ]:
# 7. Agente Revisor
revisor = Agent(
    role='Revisor de Conteúdo',
    goal='Revisar todo o conteúdo produzido, incluir links das imagens geradas e entregar a versão final ao usuário',
    verbose=True,
    memory=True,
    backstory=(
        "Você tem um olho afiado para detalhes, garantindo que todo o conteúdo "
        "esteja perfeito antes de ser entregue ao usuário."
    ),
    llm=gpt_4o_mini
)

#TAREFAS

In [ ]:
# 3. Tarefa de Escrita de Roteiro
tarefa_roteiro = Task(
    description=(
        "Escrever um roteiro detalhado para o vídeo sobre o tema: {tema}. "
        "O roteiro deve ser envolvente e fornecer um fluxo lógico de informações. "
        "Se necessário, especifique imagens que podem enriquecer o conteúdo. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Um roteiro completo e bem estruturado para o vídeo sobre {tema}.',
    agent=escritor_roteiro,
)

In [ ]:
# 4. Tarefa de SEO para YouTube
tarefa_seo = Task(
    description=(
        "Otimizar o título e o roteiro do vídeo para que tenha uma alta performance no YouTube. "
        "Incorporar as melhores práticas de SEO para garantir uma boa classificação e visibilidade. "
        "Crie hashtags, palavras-chaves e tags de vídeo para youtube. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Roteiro e título otimizados para o vídeo sobre {tema}, prontos para publicação no YouTube.',
    agent=especialista_seo,
)

In [ ]:
# 5. Tarefa de Criação de Prompt para DALL-E
tarefa_criacao_prompt_dalle = Task(
    description=(
        "Criar um prompt detalhado para gerar uma imagem no DALL-E "
        "conforme descrito no roteiro."
    ),
    expected_output='Um prompt de texto detalhado para geração de imagem no DALL-E.',
    agent=criador_prompt_dalle,
)

In [ ]:
# 6. Tarefa de Geração de Imagem com DALL-E
tarefa_geracao_imagem = Task(
    description=(
        "Gerar uma imagem usando o DALL-E com o prompt fornecido pelo Criador de Prompts para DALL-E."
    ),
    expected_output='Uma imagem gerada pronta para uso.',
    agent=gerador_imagens,
)


In [ ]:
# 7. Tarefa de Revisão e Entrega
tarefa_revisao = Task(
    description=(
        "Revisar todo o conteúdo produzido (título, roteiro, e otimização de SEO), "
        "incluir os links das imagens geradas e preparar a versão final para entrega ao usuário."
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Conteúdo revisado, com links das imagens inclusos, pronto para entrega ao usuário.',
    agent=revisor,
    output_file='output_teste.md'  # Configurando o output para salvar em um arquivo Markdown
)

#Crew

In [ ]:
# Formando a crew
crew = Crew(
    agents=[
        pesquisador,
        escritor_titulos,
        escritor_roteiro,
        especialista_seo,
        criador_prompt_dalle,
        gerador_imagens,
        revisor
    ],
    tasks=
    [
        tarefa_pesquisa,
        tarefa_titulos,
        tarefa_roteiro,
        tarefa_seo,
        tarefa_criacao_prompt_dalle,
        tarefa_geracao_imagem,
        tarefa_revisao
    ],
    process=Process.sequential  # Processamento sequencial das tarefas
)


In [ ]:
tema_user = input("Digite o tema do vídeo: ")

Digite o tema do vídeo: Inteligência Artificial no Futebol


In [ ]:
print(tema_user)

Inteligência Artificial no Futebol


In [ ]:
# Executando o processo com o tema escolhido
result = crew.kickoff(inputs={'tema': tema_user})



> Entering new CrewAgentExecutor chain...
Action: Search the internet  
Action Input: {"search_query": "Inteligência Artificial no Futebol"}   


Search results: Title: Clubes de futebol já começam a usar inteligência artificial para ...
Link: https://www.cnnbrasil.com.br/esportes/futebol/futebol-internacional/clubes-de-futebol-ja-comecam-a-usar-inteligencia-artificial-para-cacar-talentos/
Snippet: Uma empresa de tecnologia sediada em Londres pretende “democratizar” a identificação e a prospecção de talentos no futebol através de um ...
---
Title: Como a IA está impactando treinos e jogos de futebol | by LabJor
Link: https://medium.com/labjorfaap/como-a-ia-est%C3%A1-impactando-treinos-e-jogos-de-futebol-401e01d896bd
Snippet: A Inteligência Artificial também chegou ao futebol. Mapeamento da atuação dos jogadores, análises de melhoria de desempenho e desenvolvimento de ...
---
Title: Saiba como a inteligência artificial revoluciona o futebol na Europa
Link: https://www.correiobrazilien